In [7]:
import json
import pandas as pd
import requests

upcoming = []
for i in range(1,1000):
    url = 'https://api.themoviedb.org/3/movie/upcoming?api_key={1}&language=ko-KR&primary_release_date.gte=2022-01-01&page={0}'.format(i,'36927ad1d2817ff0fa31947e47d186fd')
    resp = requests.get(url)
    upcoming.append(json.loads(resp.content))

    if i == 10:
        break

upcoming_movies = []
for row in upcoming:
    for result in row ['results']:
        upcoming_movies.append(result)

In [8]:
upcoming_movies=pd.DataFrame(upcoming_movies)
upcoming_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              49 non-null     bool   
 1   backdrop_path      9 non-null      object 
 2   genre_ids          49 non-null     object 
 3   id                 49 non-null     int64  
 4   original_language  49 non-null     object 
 5   original_title     49 non-null     object 
 6   overview           49 non-null     object 
 7   popularity         49 non-null     float64
 8   poster_path        23 non-null     object 
 9   release_date       49 non-null     object 
 10  title              49 non-null     object 
 11  video              49 non-null     bool   
 12  vote_average       49 non-null     int64  
 13  vote_count         49 non-null     int64  
dtypes: bool(2), float64(1), int64(3), object(8)
memory usage: 4.8+ KB


In [9]:
# upcoming_movies.to_csv("../data-files/upcoming_movies.csv", header=True, index=True)

In [10]:
upcoming_movies=upcoming_movies.drop(['adult', 'backdrop_path', 'video','genre_ids'], axis=1)

In [13]:
upcoming_movies.head(2)

,id,original_language,original_title,overview,popularity,poster_path,release_date,title,vote_average,vote_count
0,598331,en,Rumble,,15.393,/mVKfyo2YQ1Sf0fKzZfc5t9HM6eK.jpg,2022-02-17,럼블,0,0
1,619803,ko,범죄도시 2,"가리봉동 소탕작전 이후 4년, 괴물형사 마석도(마동석)과 금천서 강력반이 강력한 미...",6.453,/sUK97eO2mgHqwQ2ZMLjpMvuon6M.jpg,2022-01-01,범죄도시 2,0,0


In [14]:
# 칼럼이름을 DB 칼럼이름과 동일하게 변경

upcoming_movies=upcoming_movies.rename(columns={'id':'movie_id', 'original_title':'subtitle','poster_path':'posterpath'})
upcoming_movies.head(2)

,movie_id,original_language,subtitle,overview,popularity,posterpath,release_date,title,vote_average,vote_count
0,598331,en,Rumble,,15.393,/mVKfyo2YQ1Sf0fKzZfc5t9HM6eK.jpg,2022-02-17,럼블,0,0
1,619803,ko,범죄도시 2,"가리봉동 소탕작전 이후 4년, 괴물형사 마석도(마동석)과 금천서 강력반이 강력한 미...",6.453,/sUK97eO2mgHqwQ2ZMLjpMvuon6M.jpg,2022-01-01,범죄도시 2,0,0


In [15]:
upcoming_movies.isna().sum()

movie_id              0
original_language     0
subtitle              0
overview              0
popularity            0
posterpath           26
release_date          0
title                 0
vote_average          0
vote_count            0
dtype: int64

In [17]:
upcoming_movies_ids=upcoming_movies[["movie_id"]]
upcoming_movies_ids

,movie_id
0,598331
1,619803
2,647250
3,587628
4,718685
5,866346
6,586373
7,551405
8,257094
9,869183


In [21]:
import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
cursor = conn.cursor()

# cursor.execute('DELETE FROM now_playing_movies')
error_list=[]
for movie_id in upcoming_movies_ids.values:
    try:
    # print(type(movie_id), movie_id)  # movie_id type이 nd.arrary임. 
        sql= "INSERT INTO upcoming_movies VALUES(%s)"
        cursor.execute(sql, movie_id[0]) 
    except:
        error_list.append(movie_id)
    
conn.commit()
cursor.close()
conn.close()